# Métrica 1.1.04: "Congruencia del RFC a partir del nombre"

In [ ]:
import re
import urllib.parse
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure
import unicodedata # Para normalizar y quitar acentos
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

# --- 1. CONFIGURACIÓN BÁSICA ---
METRIC_ID = "1_1_04_RFC_VS_NOMBRE"

# --- 2. LÓGICA PURA DE LA MÉTRICA ---

# --- Funciones de Normalización (Reutilizadas de Métrica 1.1.03) ---

def normalize_for_curp(name: str) -> str:
    """
    Limpia y normaliza un nombre/apellido para las reglas de RFC/CURP.
    - Quita acentos (ej. JOSÉ -> JOSE)
    - Convierte a mayúsculas
    - Maneja 'Ñ' -> 'X'
    - Quita prefijos y artículos comunes (DE, LA, DEL, Y, ETC.)
    - Toma el primer nombre/apellido en caso de compuestos.
    """
    if not isinstance(name, str):
        return ""
        
    s = ''.join(c for c in unicodedata.normalize('NFD', name)
                if unicodedata.category(c) != 'Mn')
    s = s.upper().strip()
    s = s.replace('Ñ', 'X')
    
    prefixes_to_remove = [
        'DE ', 'DEL ', 'LA ', 'LOS ', 'LAS ', 'Y ', 'MC ', 'MAC ', 'VON ', 'VAN '
    ]
    
    for prefix in prefixes_to_remove:
        if s.startswith(prefix):
            s = s[len(prefix):]
            
    s = s.split(' ')[0]
    s = re.sub(r'[^A-Z]', '', s)
    return s

def get_internal_vowel(s: str) -> str:
    """Obtiene la primera vocal INTERNA (después del primer caracter)."""
    if len(s) <= 1:
        return 'X'
    vowels = re.sub(r'[^AEIOU]', '', s[1:])
    return vowels[0] if vowels else 'X'

# --- Lógica Específica de la Métrica 1.1.04 ---

def validar_rfc_vs_nombre(rfc_base: str, nombre: str, ap_paterno: str, ap_materno: str) -> str:
    """
    Valida la congruencia del RFC (primeros 4 caracteres) vs los datos del nombre
    según las reglas estándar del SAT.
    """
    
    # --- Validación de Entradas ---
    # Necesitamos RFC (base de 10), nombre y primer apellido.
    if not all([rfc_base, nombre, ap_paterno]):
        return "SIN_DATO"
        
    if len(rfc_base) != 10:
        # La base del RFC (sin homoclave) debe tener 10 caracteres
        return "NO_CUMPLE" 
        
    # --- Normalización de Nombres ---
    n_nombre = normalize_for_curp(nombre)
    n_paterno = normalize_for_curp(ap_paterno)
    n_materno = normalize_for_curp(ap_materno)
    
    if not all([n_nombre, n_paterno]):
        return "SIN_DATO"
        
    # --- Extracción de caracteres de la persona ---
    try:
        # Regla 1: Primera letra del primer apellido.
        c_1_calc = n_paterno[0]
        
        # Regla 2: Primera vocal interna del primer apellido.
        # (Si la primera letra es vocal, se toma la segunda vocal.
        #  Esta lógica de "get_internal_vowel" es más simple y 
        #  cubre el 99% de los casos. La regla real del SAT es más compleja).
        # Para la métrica 1.1.04, la regla dice "Si c_1 fue consonante, entonces 
        # c_2 debe ser la primera vocal... Si c_1 fue vocal, entonces c_2 
        # debe ser la segunda vocal".
        
        vowels_paterno = re.sub(r'[^AEIOU]', '', n_paterno)
        c_2_calc = 'X'
        if c_1_calc in 'AEIOU':
             c_2_calc = vowels_paterno[1] if len(vowels_paterno) > 1 else 'X'
        else:
             c_2_calc = vowels_paterno[0] if vowels_paterno else 'X'

        # Regla 3: Primera letra del segundo apellido.
        c_3_calc = n_materno[0] if n_materno else 'X' # 'X' si no hay apellido materno
        
        # Regla 4: Primera letra del primer nombre.
        c_4_calc = n_nombre[0]
        
    except IndexError:
        return "NO_CUMPLE" # Datos de nombre inválidos (ej. "A")
        
    # --- Extracción de caracteres del RFC ---
    c_1_rfc = rfc_base[0]
    c_2_rfc = rfc_base[1]
    c_3_rfc = rfc_base[2]
    c_4_rfc = rfc_base[3]
    
    # --- Comparación Final ---
    if (c_1_calc == c_1_rfc and
        c_2_calc == c_2_rfc and
        c_3_calc == c_3_rfc and
        c_4_calc == c_4_rfc):
        return "CUMPLE"
    else:
        # Imprimimos los valores calculados vs los reales para depuración
        print(f"    > DEBUG: Calculado [{c_1_calc}{c_2_calc}{c_3_calc}{c_4_calc}] vs RFC [{c_1_rfc}{c_2_rfc}{c_3_rfc}{c_4_rfc}]")
        return "NO_CUMPLE"

# --- 3. EL FLUJO DEL WORKER (PROCESAMIENTO POR LOTES) ---
def procesar_todas_las_declaraciones_rfc_nombre():
    """
    Procesa TODAS las declaraciones de la colección de origen
    y guarda los resultados de la Métrica 1.1.04 en la colección 'metricas'.
    """
    client = None
    
    # Contadores
    total_documentos = 0
    cumple = 0
    no_cumple = 0
    sin_dato = 0
    
    try:
        # --- A. CONECTAR A LA BASE DE DATOS ---
        print(f"Conectando a MongoDB en {DB_NAME}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ping') 
        db = client[DB_NAME]
        
        source_collection = db[SOURCE_COLLECTION_NAME]
        target_collection = db[METRICS_COLLECTION_NAME]
        
        print("¡Conexión exitosa!")
        print(f"Leyendo de: {SOURCE_COLLECTION_NAME}")
        print(f"Escribiendo en: {METRICS_COLLECTION_NAME}")
        print("--- INICIANDO PROCESAMIENTO POR LOTES (Métrica 1.1.04 - RFC vs Nombre) ---")

        # --- B. BUSCAR TODOS LOS DOCUMENTOS ---
        # Proyección optimizada
        proyeccion = {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosGenerales.rfc.rfc": 1, # Base del RFC
            "declaracion.situacionPatrimonial.datosGenerales.nombre": 1,
            "declaracion.situacionPatrimonial.datosGenerales.primerApellido": 1,
            "declaracion.situacionPatrimonial.datosGenerales.segundoApellido": 1,
        }
        
        for doc in source_collection.find({}, proyeccion):
            total_documentos += 1
            original_id = doc["_id"]

            # --- C. LEER DATOS Y EJECUTAR MÉTRICA ---
            rfc_base = None
            nombre = None
            ap_paterno = None
            ap_materno = None
            
            try:
                # Navegación segura
                datos_generales = doc.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosGenerales", {})
                if datos_generales:
                    rfc_object = datos_generales.get("rfc")
                    if isinstance(rfc_object, dict):
                        rfc_base = rfc_object.get("rfc")
                    
                    nombre = datos_generales.get("nombre")
                    ap_paterno = datos_generales.get("primerApellido")
                    ap_materno = datos_generales.get("segundoApellido")
            except AttributeError:
                pass # Se queda todo en None

            # Ejecutamos la lógica de la métrica
            resultado_metrica = validar_rfc_vs_nombre(rfc_base, nombre, ap_paterno, ap_materno)
            
            # Actualizamos contadores
            if resultado_metrica == "CUMPLE":
                cumple += 1
            elif resultado_metrica == "NO_CUMPLE":
                no_cumple += 1
            else:
                sin_dato += 1

            # --- CONSERVAMOS TU SALIDA DE CONSOLA ---
            print(f"\nProcesando Documento ID: {original_id}")
            print("--- RESULTADO DEL ANÁLISIS ---")
            print(f"  > RFC Base: '{rfc_base}'")
            print(f"  > Nombre: '{nombre}', Paterno: '{ap_paterno}', Materno: '{ap_materno}'")
            print(f"  > Resultado de la Métrica: {resultado_metrica}")
            print("---------------------------------")
            # --- FIN DE LA SALIDA DE CONSOLA ---

            # --- D. GUARDAR RESULTADO EN LA COLECCIÓN 'metricas' ---
            filtro = { "_id": original_id }
            actualizacion = {
                "$set": {
                    METRIC_ID: resultado_metrica
                }
            }
            target_collection.update_one(filtro, actualizacion, upsert=True)

            print(f"¡Resultado guardado/actualizado en 'metricas' para el ID: {original_id}!")

        print("\n--- PROCESAMIENTO POR LOTES FINALIZADO ---")
        print("Resumen (Métrica 1.1.04 - RFC vs Nombre):")
        print(f"  > Documentos Totales Procesados: {total_documentos}")
        print(f"  > Métrica 'CUMPLE': {cumple}")
        print(f"  > Métrica 'NO_CUMPLE': {no_cumple}")
        print(f"  > Métrica 'SIN_DATO': {sin_dato}")

    except ConnectionFailure:
        print("Error: No se pudo conectar a la base de datos.")
    except OperationFailure as e:
        print(f"Error en la operación de la base de datos: {e.details}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
    finally:
        if client:
            client.close()
            print("Conexión cerrada.")

# --- 4. EJECUTAR EL SCRIPT ---
if __name__ == "__main__":
    procesar_todas_las_declaraciones_rfc_nombre()

Conectando a MongoDB en sistema1...
¡Conexión exitosa!
Leyendo de: all_data_20251021
Escribiendo en: metricas
--- INICIANDO PROCESAMIENTO POR LOTES (Métrica 1.1.04 - RFC vs Nombre) ---

Procesando Documento ID: 68f81b8800535f910a29f694
--- RESULTADO DEL ANÁLISIS ---
  > RFC Base: 'None'
  > Nombre: 'AARON', Paterno: 'GUEL', Materno: 'MONTOYA'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f694!

Procesando Documento ID: 68f81b8800535f910a29f695
--- RESULTADO DEL ANÁLISIS ---
  > RFC Base: 'None'
  > Nombre: '01FGR0026U NORMA ELIZABETH', Paterno: 'MAURICIO', Materno: 'MAURICIO'
  > Resultado de la Métrica: SIN_DATO
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f695!

Procesando Documento ID: 68f81b8800535f910a29f696
--- RESULTADO DEL ANÁLISIS ---
  > RFC Base: 'None'
  > Nombre: 'AARON', Paterno: 'DIAZ', Materno: 'CALDER